In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from jellyfish.core import Client
from jellyfish.history_loader import load_candles_history
from jellyfish import utils
from datetime import datetime

utils.disable_warnings()

pair = 'btcusdt'
end_dt = datetime.now()
frame = load_candles_history(Client(), pair, end_dt=end_dt, interval='1m', candles_num=5000)
frame.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,Close,High,Low,NumOfTrades,Open,QuoteAssetVolume,TakerBuyAssetVolume,TakerSellAssetVolume,Volume
Date,,,,,,,,,
2022-05-19 15:07:00,29429.28,29430.00,29381.62,1301.0,29405.58,2.088658e+06,49.57204,1.457471e+06,71.04282
2022-05-19 15:08:00,29451.85,29457.62,29429.28,694.0,29429.28,8.043869e+05,15.51287,4.567758e+05,27.31761
2022-05-19 15:09:00,29427.07,29453.37,29427.07,622.0,29451.84,8.515713e+05,10.53903,3.102706e+05,28.92426
2022-05-19 15:10:00,29462.75,29462.76,29410.00,1006.0,29427.08,3.047512e+06,48.67499,1.432327e+06,103.55573
2022-05-19 15:11:00,29486.41,29488.84,29462.75,945.0,29462.76,1.163060e+06,11.36740,3.351083e+05,39.45074


In [48]:
from jellyfish import indicator
import numpy as np

returns = frame.Close / frame.Open
hurst = indicator.hurst(returns, kind=indicator.HURST_CHANGE, simplified=False)

  0%|          | 0/4721 [00:00<?, ?it/s]

In [69]:
th = hurst[100:].mean()
th_eps = hurst[100:].std() * 1e-3
# th_eps = hurst[100:].std() * 2.5e-3

frame['Persistent'] = (hurst > th + th_eps).astype(np.int32)
frame['Reversal'] = (hurst < th - th_eps).astype(np.int32)
frame['RandomWalk'] = (np.abs(hurst - th) < th_eps).astype(np.int32)
frame['i_Hurst'] = hurst

In [70]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG

condition = lambda ohlc: ohlc.Persistent.max() + ohlc.Reversal.max() > 1
frame1 = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG)
utils.plot_ohlc(frame1)

  0%|          | 0/4820 [00:00<?, ?it/s]

Row(id='10622', ...)

In [68]:
utils.plot_ohlc(frame);